In [4]:
!pip3 install tueplots==0.0.5
!pip3 install sentence-transformers==2.2.2
!pip3 install datasets
!pip3 install accelerate -U

In [ ]:
import os
import pandas as pd

from sentence_transformers import SentenceTransformer

In [5]:
from google.colab import drive
drive.mount('/content/drive')
#CHANGE THIS PATH FOR YOU
%cd '/content/drive/MyDrive/Colab Notebooks/ML Twitter Project/Project Tutorials'
# %cd '/content/drive/MyDrive/coreML23/ML_2023/students'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ML Twitter Project/Project Tutorials'
/content


In [ ]:
team_id = '36' #put your team id here
split = 'test_1' # replace by 'test_2' for FINAL submission

df = pd.read_csv('tweets_train.csv') # .sample(100) # we are subsampling 100 training data
df_test = pd.read_csv(f'tweets_{split}.csv') # .sample(200) # we are subsampling 200 test data


In [ ]:
### PREPROCESSING IS ALREADY DONE AND WE GET WORDS COLUMN ###

# joining list of words to create sentences

df['words_str'] = df['words'].apply(lambda words: ' '.join(eval(words)))
df_test['words_str'] = df_test['words'].apply(lambda words: ' '.join(eval(words)))


In [ ]:
from datasets import Dataset
import numpy as np

df_train, df_val, df_check_test = \
              np.split(df.sample(frac=1, random_state=42),
                       [int(.8*len(df)), int(.9*len(df))])

In [ ]:
raw_train_ds = Dataset.from_pandas(df_train)
raw_val_ds = Dataset.from_pandas(df_val)
raw_test_ds = Dataset.from_pandas(df_check_test)

# This line prints the description of train_ds
raw_train_ds, raw_val_ds, raw_test_ds

(Dataset({
     features: ['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str', '__index_level_0__'],
     num_rows: 6400
 }),
 Dataset({
     features: ['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str', '__index_level_0__'],
     num_rows: 800
 }),
 Dataset({
     features: ['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str', '__index_level_0__'],
     num_rows: 800
 }))

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader

BASE_MODEL = "bert-base-uncased"
LEARNING_RATE = 2e-5
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = 36

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

def preprocess_function(examples):
    label = examples["score_compound"]
    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Change this to real number
    examples["label"] = float(label)
    return examples

for split in ds:
    ds[split] = ds[split].map(preprocess_function, remove_columns=['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str'])

ds

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

{'train': Dataset({
     features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 6400
 }),
 'validation': Dataset({
     features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 800
 }),
 'test': Dataset({
     features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 800
 })}

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [ ]:
from transformers import TrainingArguments, Trainer
import accelerate

training_args = TrainingArguments(
    output_dir="../models/bert-base-uncased-fine-tuned-regression",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

In [ ]:
import torch

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics_for_regression,
    model_init=model_init
)

#trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:357: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(


In [ ]:
#trainer.eval_dataset=ds["test"]
#trainer.evaluate()

In [ ]:
#trainer.save_model()

Gradio

In [ ]:
!pip install ray[tune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.2 MB/s eta 0:00:00


In [ ]:
from ray import tune

tune_config = {
        "num_train_epochs": tune.choice([4]),
        "batch_size": tune.choice([4,8,16,32])
    }

best_trial = trainer.hyperparameter_search(
    hp_space=lambda _: tune_config,
    direction="minimize",
    backend="ray",
    n_trials=10, # number of trials
)

2023-08-17 04:47:10,592	INFO worker.py:1621 -- Started a local Ray instance.
2023-08-17 04:47:20,303	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2023-08-17 04:47:20,382	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
/usr/local/lib/python3.10/dist-packages/ray/tune/trainable/function_trainable.py:624: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/latest/tun

+-------------------------------------------------------------------+
| Configuration for experiment     _objective_2023-08-17_04-47-20   |
+-------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator            |
| Scheduler                        FIFOScheduler                    |
| Number of trials                 10                               |
+-------------------------------------------------------------------+

View detailed results here: /root/ray_results/_objective_2023-08-17_04-47-20
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/_objective_2023-08-17_04-47-20`

Trial status: 10 PENDING
Current time: 2023-08-17 04:47:20. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+-----------------------------------------------------------------------+
| Trial name               status       num_train_epochs     batch_size |
+-----------------------------------

(pid=56639) 2023-08-17 04:48:00.456728: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Trial _objective_264cb_00000 started with configuration:
+--------------------------------------------+
| Trial _objective_264cb_00000 config        |
+--------------------------------------------+
| batch_size                              16 |
| num_train_epochs                         4 |
+--------------------------------------------+

Trial _objective_264cb_00001 started with configuration:
+--------------------------------------------+
| Trial _objective_264cb_00001 config        |
+--------------------------------------------+
| batch_size                              32 |
| num_train_epochs                         4 |
+--------------------------------------------+



(_objective pid=56639) Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
(pid=56649) 2023-08-17 04:48:00.470115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(_objective pid=56639) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
(_objective pid=56639) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(_objective pid=56639) /usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(_objective pid=56639)   warnings.warn(
  0%|          | 0/1600 [00:00<?,

Trial status: 2 RUNNING | 8 PENDING
Current time: 2023-08-17 04:48:21. Total running time: 1min 0s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
+-----------------------------------------------------------------------+
| Trial name               status       num_train_epochs     batch_size |
+-----------------------------------------------------------------------+
| _objective_264cb_00000   RUNNING                     4             16 |
| _objective_264cb_00001   RUNNING                     4             32 |
| _objective_264cb_00002   PENDING                     4              4 |
| _objective_264cb_00003   PENDING                     4             16 |
| _objective_264cb_00004   PENDING                     4             16 |
| _objective_264cb_00005   PENDING                     4             32 |
| _objective_264cb_00006   PENDING                     4              4 |
| _objective_264cb_00007   PENDING                     4              4 |
| _objective_264cb_00008   PENDING        

RuntimeError: ignored

In [ ]:
print(best_trial)

Model Save Falanları

In [ ]:
raw_test_final = Dataset.from_pandas(df_test)

ds_test = {"test": raw_test_final}

def preprocess_function(examples):
    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    return examples

for split in ds_test:
    ds_test[split] = ds_test[split].map(preprocess_function, remove_columns=['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'words_str'])

ds_test

In [ ]:
import math
import pandas as pd

y_preds = []

for i in range(len(raw_test_final)):
    input_texts = raw_test_final[i]["text"]
    encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=256, return_tensors="pt").to("cuda")
    y_preds += model(**encoded).logits.reshape(-1).tolist()

y_preds

In [ ]:
y_hat = np.array(y_preds)
y_hat.shape

In [ ]:
folder = '/content/drive/MyDrive/Colab Notebooks/ML Twitter Project/Project Tutorials'

np.save(os.path.join(folder, f'{team_id}__{split}__reg_pred.npy'), y_hat)

In [ ]:
import numpy as np
y_hat = np.array([-0.603107750415802,
 -0.5866397619247437,
 0.04154566675424576,
 0.6357582211494446,
 0.13295868039131165,
 0.8244520425796509,
 0.5955035090446472,
 0.5686573386192322,
 0.6664959192276001,
 0.13987267017364502,
 0.29588624835014343,
 -0.19705525040626526,
 0.6233379244804382,
 -0.16692529618740082,
 0.2597420811653137,
 0.215000718832016,
 0.4579668939113617,
 0.49183452129364014,
 0.08842920511960983,
 0.7876425385475159,
 0.5674670338630676,
 0.26631882786750793,
 0.12836439907550812,
 0.22015437483787537,
 0.28829488158226013,
 -0.027036841958761215,
 -0.14311425387859344,
 -0.046550702303647995,
 -0.6923142671585083,
 0.20266270637512207,
 0.8847103714942932,
 0.8634341359138489,
 0.03773428872227669,
 -0.01593487150967121,
 0.087130606174469,
 -0.30372342467308044,
 -0.25721612572669983,
 0.0404321663081646,
 0.5215537548065186,
 -0.01635020039975643,
 0.5884630084037781,
 0.6243064403533936,
 0.0783337950706482,
 0.13369973003864288,
 0.5383656620979309,
 -0.11142350733280182,
 0.7618085741996765,
 0.648409366607666,
 0.6227404475212097,
 -0.0014906334690749645,
 0.0038692420348525047,
 -0.22339041531085968,
 0.024082401767373085,
 0.04321654886007309,
 -0.34649449586868286,
 0.2932969629764557,
 0.5861774682998657,
 0.3926283121109009,
 0.13111598789691925,
 -0.16081421077251434,
 0.456136018037796,
 -0.5406923890113831,
 0.05086585134267807,
 0.24352829158306122,
 0.4616487920284271,
 0.6693354249000549,
 0.29488494992256165,
 0.01500740833580494,
 -0.2243107557296753,
 0.8085107207298279,
 0.9433306455612183,
 0.6426016092300415,
 -0.020645922049880028,
 0.22072872519493103,
 0.06846383959054947,
 0.6026651263237,
 -0.6899497509002686,
 -0.003698941320180893,
 0.8809376955032349,
 -0.12776756286621094,
 0.01585380733013153,
 0.010544117540121078,
 -0.23637086153030396,
 0.25377872586250305,
 0.10964395105838776,
 0.8446930050849915,
 0.04717785120010376,
 0.36136990785598755,
 0.8542171716690063,
 0.028008975088596344,
 0.8234671950340271,
 0.27268102765083313,
 0.5055124759674072,
 0.41091907024383545,
 -0.0059205335564911366,
 0.2867012321949005,
 -0.028911886736750603,
 0.02658579871058464,
 0.3068651556968689,
 0.5699227452278137,
 0.41577962040901184,
 0.28238725662231445,
 0.5064287185668945,
 0.3518364131450653,
 0.5115510821342468,
 0.9845372438430786,
 0.6349045634269714,
 0.2957550585269928,
 -0.08816292881965637,
 -0.1241394653916359,
 -0.2140994369983673,
 0.013800585642457008,
 -0.02186386100947857,
 0.7478705048561096,
 -0.45388874411582947,
 0.06550613790750504,
 -0.03572431206703186,
 0.5703895688056946,
 -0.39414873719215393,
 0.9866748452186584,
 0.25588804483413696,
 0.8289502859115601,
 0.9173116087913513,
 0.7918393015861511,
 0.5994880795478821,
 0.06229724362492561,
 0.4993251860141754,
 0.7007018327713013,
 0.6515277624130249,
 -0.01539161428809166,
 -0.42993733286857605,
 0.3224649429321289,
 0.6447791457176208,
 0.6463728547096252,
 0.26953867077827454,
 0.4384189546108246,
 0.17855115234851837,
 0.13464468717575073,
 0.03840743750333786,
 -0.44327160716056824,
 0.42531511187553406,
 -0.20702174305915833,
 -0.002104031853377819,
 0.3221285045146942,
 -0.1337766945362091,
 0.31520751118659973,
 0.3275783956050873,
 0.41530510783195496,
 0.7161596417427063,
 -0.28205856680870056,
 -0.045587267726659775,
 -0.1663409322500229,
 -0.2270711213350296,
 -0.14023657143115997,
 0.019910797476768494,
 0.7267714142799377,
 0.00662505766376853,
 0.57093745470047,
 0.1850271373987198,
 0.4012569487094879,
 -0.3853164613246918,
 0.9060755968093872,
 0.7693381309509277,
 0.08232954889535904,
 -0.359260231256485,
 -0.4111449718475342,
 0.6479641795158386,
 0.1303166151046753,
 0.4153464138507843,
 -0.030164990574121475,
 0.7900145053863525,
 0.7286059260368347,
 0.056284788995981216,
 -0.06921457499265671,
 0.7813475131988525,
 -0.5892935395240784,
 0.6824608445167542,
 0.2064920961856842,
 0.057675376534461975,
 0.47490963339805603,
 0.18601970374584198,
 0.7389717698097229,
 0.0370166189968586,
 0.5682759284973145,
 -0.4006298780441284,
 0.04617282375693321,
 0.30091145634651184,
 0.8781417012214661,
 -0.003954102750867605,
 0.13710764050483704,
 0.09726986289024353,
 0.662291407585144,
 0.5157536268234253,
 -0.5042800307273865,
 0.2362833023071289,
 0.6099011898040771,
 -0.23293446004390717,
 0.3246326148509979,
 -0.0879482701420784,
 0.015407640486955643,
 0.21466685831546783,
 0.6438178420066833,
 -0.23271387815475464,
 0.5540984272956848,
 -0.02363334409892559,
 -0.06917665898799896,
 -0.2873048186302185,
 0.10785230249166489,
 0.7148867845535278,
 0.9003130793571472,
 -0.14625492691993713,
 -0.7164900302886963,
 -0.030165251344442368,
 0.8401395678520203,
 0.011802767403423786,
 0.2145671546459198,
 0.2513459324836731,
 0.8544291257858276,
 0.4386311173439026,
 0.42569205164909363,
 -0.09747536480426788,
 -0.29016348719596863,
 -0.3393608033657074,
 0.8681063652038574,
 -0.2653908431529999,
 0.6543182730674744,
 0.03247052803635597,
 -0.6156600117683411,
 0.5193908214569092,
 0.7678362727165222,
 0.007446496281772852,
 0.1786414086818695,
 0.5951061248779297,
 0.07119165360927582,
 0.255725234746933,
 -0.0445958711206913,
 0.7799368500709534,
 0.8216990828514099,
 0.2967407703399658,
 0.4453446865081787,
 0.3818659484386444,
 0.8941874504089355,
 0.8146164417266846,
 0.5500757694244385,
 0.2660791873931885,
 0.6353506445884705,
 0.16627205908298492,
 0.9321740865707397,
 -0.2527855336666107,
 0.4358748197555542,
 -0.650981068611145,
 0.7106649279594421,
 0.25025537610054016,
 0.639518141746521,
 0.5409310460090637,
 0.17474056780338287,
 0.910335123538971,
 0.5060177445411682,
 0.7836417555809021,
 0.3457392454147339,
 0.6751112341880798,
 0.6483035087585449,
 -0.23205822706222534,
 -0.5583513379096985,
 0.005286879371851683,
 0.05869261175394058,
 -0.02654382772743702,
 -0.01841191202402115,
 0.28084567189216614,
 0.39001521468162537,
 -0.3815027177333832,
 0.8679128289222717,
 0.3520714044570923,
 0.8541023135185242,
 -0.004136048723012209,
 0.5406116247177124,
 -0.07714617997407913,
 -0.01725981943309307,
 0.38187894225120544,
 0.03397628292441368,
 0.16955089569091797,
 0.5373619794845581,
 0.4673413038253784,
 0.5265792608261108,
 0.23527421057224274,
 -0.049710605293512344,
 0.544514000415802,
 0.08654815703630447,
 0.4710618555545807,
 0.020817114040255547,
 0.01933537982404232,
 0.8019371628761292,
 -0.03358597680926323,
 0.3072018325328827,
 0.4602952301502228,
 0.15162065625190735,
 0.03982885554432869,
 0.3610388934612274,
 0.4775526821613312,
 0.11373907327651978,
 0.5196018815040588,
 0.7131247520446777,
 0.721104085445404,
 0.3996998369693756,
 0.19448423385620117,
 0.8602136969566345,
 0.5661326050758362,
 0.6822513341903687,
 0.250079482793808,
 0.839101254940033,
 0.2274385690689087,
 0.2187795788049698,
 0.053118836134672165,
 -0.09661974757909775,
 0.9820742011070251,
 -0.10191288590431213,
 0.05415135622024536,
 0.21229632198810577,
 0.010198970325291157,
 -0.7021204233169556,
 0.19569867849349976,
 0.23975419998168945,
 0.5187599658966064,
 -0.013379612937569618,
 0.46395882964134216,
 0.5697475075721741,
 0.03512183576822281,
 0.01062309555709362,
 0.49301978945732117,
 0.6415865421295166,
 0.8357363343238831,
 0.5296837687492371,
 -0.018326761201024055,
 0.3728252053260803,
 -0.4443053901195526,
 0.06571739166975021,
 0.4612846076488495,
 0.18772564828395844,
 -0.42119258642196655,
 -0.45498839020729065,
 0.15983839333057404,
 0.0375470407307148,
 0.47682321071624756,
 -0.2502545714378357,
 -0.11345884203910828,
 0.45254310965538025,
 0.5010636448860168,
 0.13313080370426178,
 0.7574055790901184,
 0.03289925679564476,
 0.5319394469261169,
 0.3622419238090515,
 -0.6615593433380127,
 -0.04234844818711281,
 -0.2502191364765167,
 -0.4022206664085388,
 0.2576988637447357,
 0.7257409691810608,
 -0.4470599293708801,
 0.8181197643280029,
 0.04066150262951851,
 0.5640186071395874,
 -0.009192107245326042,
 0.6849243640899658,
 -0.3327481150627136,
 0.28317585587501526,
 0.3106970489025116,
 -0.07060930877923965,
 0.7247074246406555,
 0.5773314833641052,
 -0.5040888786315918,
 0.21269048750400543,
 -0.0471477247774601,
 -0.3169897198677063,
 -0.045737236738204956,
 0.19603939354419708,
 0.18458372354507446,
 0.8514946103096008,
 0.04135170206427574,
 -0.48839107155799866,
 0.5003716945648193,
 -0.05803205817937851,
 0.16644200682640076,
 0.5573058724403381,
 -0.4822363555431366,
 -0.5435923933982849,
 0.10717352479696274,
 0.23299100995063782,
 0.09700159728527069,
 0.654398500919342,
 -0.38557174801826477,
 0.2711981236934662,
 0.257320761680603,
 -0.4391096532344818,
 0.012596781365573406,
 0.05212127044796944,
 -0.020087987184524536,
 -0.10828390717506409,
 0.4241630733013153,
 -0.1357303112745285,
 -0.6261067986488342,
 0.8696324229240417,
 -0.036533188074827194,
 -0.4874921441078186,
 -0.00839991495013237,
 0.48289045691490173,
 0.8439475297927856,
 -0.14147396385669708,
 0.3312014639377594,
 0.4825011193752289,
 0.9148696064949036,
 -0.35886791348457336,
 -0.034060996025800705,
 0.1294892579317093,
 0.5324438810348511,
 0.6639336347579956,
 0.7053993344306946,
 0.5059345364570618,
 0.16830992698669434,
 0.40633198618888855,
 -0.20858553051948547,
 0.4743477404117584,
 -0.4530266523361206,
 -0.0735122337937355,
 0.4129287898540497,
 0.1000818982720375,
 -0.010895885527133942,
 0.19079898297786713,
 -0.009104147553443909,
 0.4896010458469391,
 0.2899329364299774,
 0.3285079002380371,
 0.0994848981499672,
 0.6746379137039185,
 -0.26070815324783325,
 0.35784944891929626,
 0.8263963460922241,
 0.36222270131111145,
 0.5026026964187622,
 0.26888999342918396,
 -0.4777701795101166,
 -0.4841808080673218,
 0.4868302345275879,
 -0.00011650431406451389,
 0.5987507700920105,
 -0.23659348487854004,
 0.025121016427874565,
 0.5054665207862854,
 -0.6792321801185608,
 -0.34521546959877014,
 -0.017108233645558357,
 -0.08269280195236206,
 0.08087047189474106,
 0.002885512076318264,
 0.01091817021369934,
 -0.21762359142303467,
 0.03729680925607681,
 -0.2870873808860779,
 0.8824421763420105,
 -0.7584007382392883,
 0.2569424510002136,
 0.4754093587398529,
 -0.3907603621482849,
 0.15024295449256897,
 0.27843600511550903,
 0.5271674990653992,
 0.27185916900634766,
 -0.0780678242444992,
 0.37554261088371277,
 0.4507594704627991,
 0.6228138208389282,
 0.2926498353481293,
 0.529658854007721,
 0.1695195734500885,
 -0.051493458449840546,
 0.7359904646873474,
 0.01721297949552536,
 -0.452676385641098,
 0.6514334082603455,
 -0.021361980587244034,
 -0.3126660883426666,
 0.38979092240333557,
 0.5985518097877502,
 0.028577476739883423,
 -0.027852777391672134,
 0.9586353898048401,
 0.6290961503982544,
 0.9383617639541626,
 -0.1575462520122528,
 0.02626728266477585,
 0.007523649837821722,
 0.6215026378631592,
 0.8850839138031006,
 0.022814374417066574,
 -0.09727636724710464,
 0.19422796368598938,
 0.10307634621858597,
 0.6961597204208374,
 -0.2123328298330307,
 0.3750948905944824,
 0.7169012427330017,
 -0.6039827466011047,
 0.7305715680122375,
 0.8775650858879089,
 0.2727675139904022,
 0.43311795592308044,
 0.668829619884491,
 0.07453729212284088,
 0.37226614356040955,
 0.7686610817909241,
 0.6272822618484497,
 0.5546016097068787,
 0.13817915320396423,
 0.9316677451133728,
 -0.22243107855319977,
 0.7930260300636292,
 0.43082448840141296,
 0.5172261595726013,
 0.7979748249053955,
 0.3699330985546112,
 0.598251223564148,
 -0.2833442687988281,
 0.05411470681428909,
 0.6352730989456177,
 0.1647166609764099,
 0.2161332368850708,
 0.5426470637321472,
 -5.124467861605808e-05,
 0.14361253380775452,
 0.4753224551677704,
 0.018445661291480064,
 0.20084983110427856,
 -0.40010184049606323,
 0.3018374741077423,
 0.03846465423703194,
 -0.15336698293685913,
 0.17752331495285034,
 0.5883116126060486,
 -0.4283181130886078,
 0.8219522833824158,
 0.4176599681377411,
 0.6669189929962158,
 0.7113933563232422,
 0.2728555202484131,
 0.6816897392272949,
 0.008145994506776333,
 0.44288936257362366,
 -0.010183227248489857,
 -0.35173919796943665,
 0.47779321670532227,
 -0.004338617902249098,
 0.06889017671346664,
 -0.04499175027012825,
 0.6492469310760498,
 0.7580756545066833,
 0.3022247552871704,
 -0.07162760943174362,
 0.22945377230644226,
 -0.16113132238388062,
 0.9016544222831726,
 0.7245451807975769,
 0.7072034478187561,
 -0.08082108199596405,
 -0.4100878834724426,
 0.2654060423374176,
 -0.45070019364356995,
 -0.09408726543188095,
 0.052136849611997604,
 0.3949072062969208,
 0.7408193945884705,
 -0.077904611825943,
 0.38735952973365784,
 0.9714735150337219,
 0.21089160442352295,
 0.12905530631542206,
 0.3366906940937042,
 -0.3259265422821045,
 -0.919982373714447,
 0.9727023243904114,
 0.7810806632041931,
 0.5084410905838013,
 0.5625805258750916,
 0.12523731589317322,
 0.15134090185165405,
 0.629491925239563,
 0.31680601835250854,
 0.0243709497153759,
 0.16088658571243286,
 -0.02163982018828392,
 0.33137965202331543,
 0.11097215861082077,
 0.43141576647758484,
 -0.45895445346832275,
 0.7521007657051086,
 0.4576728641986847,
 0.17016997933387756,
 0.3561089038848877,
 -0.011157020926475525,
 0.7912461161613464,
 0.6092283129692078,
 0.5288174152374268,
 0.02431165985763073,
 0.6030229926109314,
 -0.6516829133033752,
 -0.4101158082485199,
 0.9422385096549988,
 -0.0648704394698143,
 0.023547930642962456,
 0.4229373633861542,
 0.10501235723495483,
 0.6007576584815979,
 0.08922723680734634,
 0.3630581498146057,
 0.45086726546287537,
 0.0567498616874218,
 -0.5026940107345581,
 -0.07672601193189621,
 0.8636863827705383,
 0.5616263151168823,
 -0.12853258848190308,
 0.6045470237731934,
 0.2521131932735443,
 0.012159925885498524,
 -0.3851925730705261,
 0.4349534213542938,
 -0.47555670142173767,
 0.617259681224823,
 0.7133774161338806,
 0.048214733600616455,
 0.10692397505044937,
 0.8079749941825867,
 0.3085809350013733,
 0.7274170517921448,
 -0.003705420531332493,
 0.3115265965461731,
 0.9106112122535706,
 0.12818673253059387,
 0.07038772851228714,
 0.9435996413230896,
 0.43284374475479126,
 0.3090018332004547,
 -0.03458341956138611,
 -0.601147472858429,
 0.5379806160926819,
 0.05908525362610817,
 -0.15598665177822113,
 0.7381206154823303,
 -0.009109890088438988,
 -0.7340116500854492,
 0.47733473777770996,
 0.7425679564476013,
 0.07517826557159424,
 0.5994934439659119,
 0.18150094151496887,
 0.45727255940437317,
 0.4462089240550995,
 -0.15916991233825684,
 -0.006117114331573248,
 0.36581236124038696,
 0.10791191458702087,
 -0.44948554039001465,
 0.4740932285785675,
 0.6451225876808167,
 0.7024200558662415,
 -0.21271827816963196,
 0.5440874695777893,
 0.5135996341705322,
 0.35361573100090027,
 0.022251833230257034,
 0.7837496995925903,
 0.8909288644790649,
 0.909631073474884,
 0.3551923930644989,
 -0.2996172606945038,
 -0.08218581974506378,
 -0.2916761636734009,
 0.8017076253890991,
 0.5455283522605896,
 -0.5867723822593689,
 0.020760979503393173,
 0.5518046021461487,
 -0.19600015878677368,
 -0.0006821747520007193,
 -0.023277506232261658,
 0.8783635497093201,
 -0.01103544607758522,
 0.4750387966632843,
 0.0001618069945834577,
 -0.0015545932110399008,
 0.8365288376808167,
 0.4377090632915497,
 -0.4129893183708191,
 0.9134126901626587,
 0.7039147019386292,
 0.21078139543533325,
 0.14120852947235107,
 0.11911248415708542,
 0.6846775412559509,
 0.9163233637809753,
 0.6826016902923584,
 0.7835714221000671,
 0.12284822762012482,
 -0.022429533302783966,
 0.6749837398529053,
 -0.08355244249105453,
 0.25194376707077026,
 0.6421329379081726,
 0.7327471971511841,
 0.4721969962120056,
 0.8983690738677979,
 -0.34843358397483826,
 0.8264220356941223,
 0.3840790092945099,
 0.018201284110546112,
 0.06123209744691849,
 -0.36929911375045776,
 0.458139032125473,
 0.8955425024032593,
 0.8902252316474915,
 0.8519073128700256,
 -0.241953045129776,
 0.6611416935920715,
 -0.29240748286247253,
 0.0010632621124386787,
 -0.014676539227366447,
 0.8454945683479309,
 0.33649030327796936,
 0.5524804592132568,
 0.3132028579711914,
 0.07873541861772537,
 -0.3289519250392914,
 0.7866145968437195,
 0.17580699920654297,
 0.019417818635702133,
 1.004457950592041,
 0.08755841851234436,
 -0.3651627004146576,
 0.86618971824646,
 -0.4458158016204834,
 -0.24908749759197235,
 -0.22311213612556458,
 -0.005519466008991003,
 0.9641216397285461,
 -0.6337627172470093,
 -0.22134269773960114,
 -0.1458098590373993,
 0.2790290415287018,
 0.5465940237045288,
 -0.38209065794944763,
 -0.005055600311607122,
 0.4242611825466156,
 -0.6489752531051636,
 0.32456502318382263,
 -0.653419017791748,
 0.3690774142742157,
 0.8455380797386169,
 -0.12389817088842392,
 0.18024007976055145,
 -0.02228466048836708,
 0.042957205325365067,
 -0.1388929784297943,
 0.7708709239959717,
 0.8565679788589478,
 0.4380241632461548,
 0.20475512742996216,
 0.3097054660320282,
 0.09659119695425034,
 -0.12661553919315338,
 0.04189296439290047,
 0.1419087052345276,
 0.4706251919269562,
 0.13303640484809875,
 0.312828928232193,
 0.2350134402513504,
 -0.04059319570660591,
 -0.388534277677536,
 -0.7198060154914856,
 -0.40606972575187683,
 -0.3064602315425873,
 0.6721424460411072,
 -0.08823216706514359,
 0.7338029742240906,
 -0.004180183168500662,
 -0.007502266671508551,
 0.852832019329071,
 0.5283320546150208,
 0.5283436179161072,
 0.09842632710933685,
 0.0009400243288837373,
 -0.06272456049919128,
 0.9051331877708435,
 0.3910316824913025,
 0.00844181515276432,
 0.12598824501037598,
 -0.00879327580332756,
 0.6505610346794128,
 0.7652733325958252,
 -0.07758862525224686,
 0.07088639587163925,
 0.45866701006889343,
 0.0006343540153466165,
 0.056473568081855774,
 0.27642011642456055,
 0.8553763628005981,
 0.23677611351013184,
 -0.22671861946582794,
 0.5365607738494873,
 0.13313201069831848,
 0.7368801236152649,
 0.3215070068836212,
 0.735381543636322,
 -0.06795240193605423,
 0.035632211714982986,
 0.941417396068573,
 -0.0033720890060067177,
 -0.01530245691537857,
 0.33701738715171814,
 0.5964186191558838,
 0.8262513875961304,
 0.0162845216691494,
 -0.057805951684713364,
 -0.04374932125210762,
 -0.6857838034629822,
 -0.33192533254623413,
 -0.1640997678041458,
 0.3759000599384308,
 0.534137487411499,
 0.4967006742954254,
 0.4082820415496826,
 0.019796788692474365,
 0.39800071716308594,
 -0.02791968733072281,
 0.6603279113769531,
 0.6674923896789551,
 0.004347093869000673,
 0.8003000617027283,
 0.45923128724098206,
 -0.3081459403038025,
 0.19163165986537933,
 0.08974464982748032,
 0.8923031687736511,
 0.5030904412269592,
 0.005863414611667395,
 0.40940341353416443,
 0.9221484661102295,
 0.42883917689323425,
 -0.0009995063301175833,
 -0.39308860898017883,
 -0.2590820789337158,
 0.46166756749153137,
 -0.4911411702632904,
 -0.05119846388697624,
 -0.3454391658306122,
 -0.01265482697635889,
 0.25850266218185425,
 0.43348848819732666,
 0.4510003626346588,
 0.4415067136287689,
 -0.06524661928415298,
 -0.034252945333719254,
 0.6291730999946594,
 -0.08485931903123856,
 0.010515222325921059,
 0.5723533034324646,
 -0.03775544837117195,
 -0.07742556184530258,
 0.508741021156311,
 0.2977467179298401,
 -0.03412053361535072,
 0.019466176629066467,
 0.9011891484260559,
 0.021984998136758804,
 0.10602856427431107,
 0.3439927399158478,
 -0.11672064661979675,
 0.5026373267173767,
 0.5216342806816101,
 0.020872771739959717,
 0.23243407905101776,
 0.31708651781082153,
 0.5305530428886414,
 0.08317526429891586,
 0.40892890095710754,
 -0.19895325601100922,
 0.522948145866394,
 -0.0010108700953423977,
 0.15766312181949615,
 -0.2917163074016571,
 0.0869041159749031,
 0.6263747811317444,
 0.01338617131114006,
 0.014720569364726543,
 0.09330851584672928,
 0.055228251963853836,
 0.052631162106990814,
 -0.7169564366340637,
 0.07207411527633667,
 0.9435997605323792,
 0.610034167766571,
 0.3573242425918579,
 0.0003925901255570352,
 0.032496046274900436,
 0.20369106531143188,
 0.5933916568756104,
 0.04243175685405731,
 0.5239803791046143,
 0.16103485226631165,
 0.02098185010254383,
 0.8900635838508606,
 0.47260695695877075,
 0.16381153464317322,
 -0.06884565204381943,
 0.7751168012619019,
 0.543021023273468,
 0.6827033758163452,
 0.8821342587471008,
 0.7804594039916992,
 0.6434154510498047,
 0.4327145218849182,
 -0.13723790645599365,
 0.3540342152118683,
 -0.004933060612529516,
 -0.22931046783924103,
 0.8191658854484558,
 0.74200439453125,
 0.5203938484191895,
 0.7875135540962219,
 -0.02842658944427967,
 0.49895989894866943,
 0.684476912021637,
 0.8366774916648865,
 0.49615174531936646,
 -0.2572527527809143,
 -0.003305246587842703,
 0.5792179107666016,
 -0.22379425168037415,
 0.0862322747707367,
 0.1543857902288437,
 -0.49637752771377563,
 0.05141960456967354,
 0.582325279712677,
 -0.49325031042099,
 0.93742436170578,
 0.7270656228065491,
 -0.17020873725414276,
 0.7842593193054199,
 -0.07021916657686234,
 -0.11354465782642365,
 0.051523156464099884,
 0.08725588768720627,
 0.5489658713340759,
 -0.32362982630729675,
 -0.36192336678504944,
 -0.09561668336391449,
 0.014153877273201942,
 0.02912576124072075,
 0.4252716302871704,
 0.8046947121620178,
 0.5057995915412903,
 -0.616773247718811,
 0.30222681164741516,
 -0.07623869925737381,
 -0.084403395652771,
 0.04341435059905052,
 0.0031486828811466694,
 0.6713164448738098,
 0.5630337595939636,
 0.7247563600540161,
 0.4370265007019043,
 -0.7198044657707214,
 0.3917020857334137,
 0.814350426197052,
 -0.5084134340286255,
 0.26511242985725403,
 -0.295393168926239,
 0.9006547927856445,
 0.08775600045919418,
 -0.16419395804405212,
 0.06402882188558578,
 -0.5354148745536804,
 0.5343111157417297,
 -0.5870417952537537,
 0.12685881555080414,
 -0.04877455160021782,
 -0.030339544638991356,
 0.33846285939216614,
 -0.12215261161327362,
 0.6930833458900452,
 0.018195001408457756,
 0.858472466468811,
 0.005159773398190737,
 0.0010929065756499767,
 0.8164835572242737,
 0.6214808821678162,
 -0.007994087412953377,
 0.33348581194877625,
 -0.050397373735904694,
 0.5550085306167603,
 -0.08333002030849457,
 0.5536280274391174])

folder = '/content/drive/MyDrive/Colab Notebooks/ML Twitter Project/Project Tutorials'
import os
team_id = 36
split="test_1"
from google.colab import drive
drive.mount('/content/drive')
#CHANGE THIS PATH FOR YOU
%cd '/content/drive/MyDrive/Colab Notebooks/ML Twitter Project/Project Tutorials'
# %cd '/content/drive/MyDrive/coreML23/ML_2023/students'

np.save(os.path.join(folder, f'lol__{team_id}__{split}__reg_pred.npy'), y_hat)